## Ridge

Following [the documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html), it performs a regression trying to minimize the sum of squared error (as the linear regression above does) with a penalty on the size of the coefficients. Therefore, it tries to minimize the following quantity
$$ ||y - Xw||^2_2 + \alpha  ||w||^2_2 , $$ 
where $w$ is the coefficient vector and the parameter $\alpha$ controls the amount of regularization (called $L_2$ as it uses the $L_2$ norm of the coefficient vector). The higher this parameter, the lower will be the variance of the model.

It is a very useful regularization in case of multicollinearity in the input data that tipically lead to very large coefficients. The regularization is putting a penalty on larger coefficients, thus reducing the effect of multicollinearity.

On the contrary of the simple linear regression, we have some hyperparameters to play with

In [ ]:
import pandas as pd
import numpy as np

import random

import source.hyperplots as hyp
import source.explore as exp
from source.report import plot_predictions, make_results, store_results
from source.utility import cv_score, grid_search
import source.transf_univ as df_p

from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso, ElasticNet, SGDRegressor, BayesianRidge, Perceptron

%matplotlib inline
pd.set_option('max_columns', 500)

In [ ]:
Ridge()

* alpha: the regularization and most important hyperparameter to tune. It must be positive. When set to 0, we get the simple Linear regression.
* copy_X: that does not affect the results of the model but, if False, may lead to overwriting the input data
* fit_intercept: whether or not using an intercept term, not relevant for our cases as we, rightfully, always scale and center our data
* max_iter: a parameter that control the maximum number of iterations the solver can take to converge. Very rarely it needs to be increased and, if so, we might need to consider a different approach
* normalize: whether or not normalizing the data and it is ignored if set to False. If True, it uses the StandardScaler we already have in our pipeline
* random_state: relevant only for one of the solver since it is stochastic (the 'sag' solver)
* solver: the type of solver. Almost always leaving to `auto` is a good move as it will pick the best one for the data provided. The coiche will influence the speed and the behavior in particular situations.
* tol: the precision of the solution

## Noise and correlation

We can now repeat the previous experiments and observe how the algorithm is behaving differently even in the simplest case.

In [ ]:
def experiment_ridge(data_name, kfolds, store=False, sample=False, **kwargs):
    df = pd.read_csv(data_name)
    if sample:
        df = df.sample(sample)
    
    coefs_name = data_name.split('.csv')[0] + '__coefficients.csv'
    target_name = data_name.split('/')[2].split('.csv')[0]

    target = df['target']

    df_train = df.drop('target', axis=1)

    model = Pipeline([('processing', numeric_pipe),
                      ('scl', df_p.df_scaler()), 
                      ('ridge', Ridge(**kwargs))])

    oof, coefs_est = cv_score(df_train, target, kfolds, model, imp_coef=True)

    plot_predictions(df_train, target, oof, feature=None, hue=None, legend=False, savename=False)

    hyp.plot_coefficients('tar_nonlin_3', coefs_est, 
                          coefs_real=coefs_name)
    
    if len(kwargs.keys()) == 0:
        kwargs = {'alpha': 1}
    
    if store:
        store_results('data/03_linear_models.csv', 
                      label=target, prediction=oof, model='Ridge', 
                      parameters=kwargs, 
                      target_name=target_name, variables='All', instances=df_train.shape[0], verbose=True)
    else:
        res = make_results(label=target, prediction=oof, model='Ridge', 
                           parameters=kwargs, 
                           target_name=target_name, variables='All', instances=df_train.shape[0], verbose=True)

In [ ]:
experiment_ridge('data/simulated/10_feat_10_inf_nonoise.csv', kfolds, store=True)

While the coefficients are again estimated perfectly, the residuals show a pattern that suggests that the model is having more and more troubles in predicting larger (in the absolute sense) values of the target variable.

On the other hand, the behavior in presence of noise and/or correlation is the nearly identical

In [ ]:
experiment_ridge('data/simulated/10_feat_10_inf_noise.csv', kfolds, store=True)

In [ ]:
experiment_ridge('data/simulated/100_feat_65_inf_noise_rank.csv', kfolds, store=True)

## Number of instances

Due to the fact that some of the solvers are faster, the only difference we observe is in the speed. The training time still grows linearly, but with a lower rate of increase.

In [ ]:
def ridge_learning_curve(data_name, **kwargs):
    df = pd.read_csv(data_name)
    
    title = data_name.split('/')[2].split('.csv')[0]

    target = df['target']

    df_train = df.drop('target', axis=1)

    model = Pipeline([('processing', numeric_pipe),
                      ('scl', df_p.df_scaler()), 
                      ('ridge', Ridge(**kwargs))])

    hyp.plot_learning_curve(model, title, 
                            df_train, target, scoring='neg_mean_absolute_error')

In [ ]:
ridge_learning_curve('data/simulated/10_feat_10_inf_noise.csv')

In [ ]:
ridge_learning_curve('data/simulated/100_feat_65_inf_noise.csv')

In [ ]:
experiment_ridge('data/simulated/10_feat_10_inf_nonoise.csv', kfolds, store=True, sample=300)

For the first time we notice that the regularized model is performing better than the simple linear regression in presence of noise when the we have a limited number of instances. The better performance is visible in all the metrics and in the coefficients estimates

In [ ]:
experiment_ridge('data/simulated/10_feat_10_inf_noise.csv', kfolds, store=True, sample=300)

## Hyperparameters

All the above experiments were performed using the default hyperparameters, which generally gives a good indication of the model performance to compare results. However, this time we can control more the behavior of the model. This section will do just that.

First, we need to generate the results for different model configurations. We do so by using the custom grid search function imported from the utility module.

We will explore configurations that differ per regularization, precision, and solver type. Due to the need of seeing some pattern, we will use a dataset without noise in order to have errors at an order of magnitude that let us see the variation when the hyperparameter varies. Later, using more complex dataset, we will see more interesting patterns

In [ ]:
model = Pipeline([('processing', numeric_pipe),
                  ('scl', df_p.df_scaler()),
                  ('ridge', Ridge(random_state=235))])

param_grid = {'ridge__alpha': list(np.arange(0.05, 10, 0.05)), 
              'ridge__tol': [0.00001, 0.001, 0.1, 1], 
              'ridge__solver': ['svd', 'sparse_cg', 'lsqr', 'saga']}

df = pd.read_csv('data/simulated/100_feat_65_inf_nonoise.csv').sample(300)

target = df['target']

df_train = df.drop('target', axis=1)

res, bp, _ = grid_search(df_train, target, model, param_grid, 'neg_mean_squared_error', kfolds)

print(bp)
res.head()

First, we see how the choice of the solver is influencing the training time, with `lsqr` and `sparse_cg` being the fastest. On the other hand, different solvers seem to get different scores but it is worth noticing that, by increasing the regularization, this difference becomes less and less relevant.

In [ ]:
hyp.plot_hyperparameter(res[(res.param_ridge__alpha == 0.5) & (res.param_ridge__tol==0.001)], 'param_ridge__solver', 'Solver')

In [ ]:
hyp.plot_hyperparameter(res[(res.param_ridge__alpha == 5) & (res.param_ridge__tol==0.001)], 'param_ridge__solver', 'Solver')

This is an artifact of the role of alpha in the model performance, which in this case (a purely linear regression) is getting worse and worse when alpha gets bigger. At the same time, we also notice that this choice does not influence the training time.

In [ ]:
hyp.plot_hyperparameter(res[(res.param_ridge__solver=='lsqr') & (res.param_ridge__tol==0.001)], 'param_ridge__alpha', 'Alpha')

In [ ]:
hyp.plot_two_hyperparms(res[(res.param_ridge__solver=='lsqr')], 'param_ridge__alpha', 'param_ridge__tol', 'Alpha vs Tolerance')